# Imports and setting up viz

NB : conda lam1env (Python3.12)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *
from tools_mapping import *
from tools_hf import *
from tools_native import *
from tools_LIAISE import *
from tools_mesoNH import *

# sys.path.append('../../python_tools/aborella/PLOTS/')
# from datasets import *
sys.path.append('../../python_tools/aborella/UTIL/')
import xr_utils

In [ ]:
psy.rcParams['auto_show'] = True
mpl.rcParams['figure.figsize'] = [10., 8.]

# Load files

## LIAISE site obs

### Stations measurements

In [ ]:
#load all July obs in one ds
cen_mto_filename='../../../obs/LIAISE/LIAISE_LA-CENDROSA_CNRM_MTO-FLUX-30MIN_L2/LIAISE_LA-CENDROSA_CNRM_MTO-FLUX-30MIN_L2_2021-07*_V3.nc'
cen_mto = format_Cendrosa_obs(cen_mto_filename)
cen_mto

In [ ]:
start_time = '2021-07-14T01:00:00'
end_time = '2021-07-30T23:00:00'
restrict_cen_mto = filter_xarray_by_timestamps(cen_mto, start_time, end_time)

In [ ]:
filename =  '../../../obs/LIAISE/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2_20210721_V4.0.dat'
ds = dat_to_xarray(filename, 152, 153)
els_mto_2107 = format_ElsPlans_obs(ds, start_day='2021-07-21')

filename =  '../../../obs/LIAISE/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2_20210722_V4.0.dat'
ds = dat_to_xarray(filename, 152, 153)
els_mto_2207 = format_ElsPlans_obs(ds, start_day='2021-07-22')

In [ ]:
july_days=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
for day in july_days:
    filename =  '../../../obs/LIAISE/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2/LIAISE_ELS-PLANS_UKMO_MTO-30MIN_L2_202107{}_V4.0.dat'.format(day)
    ds = dat_to_xarray(filename, 152, 153)
    day_els_mto = format_ElsPlans_obs(ds, start_day='2021-07-{}'.format(day))
    if day=='01':
        els_mto=day_els_mto
    else:
        els_mto=xr.concat([els_mto,day_els_mto],dim='time')
els_mto

In [ ]:
restrict_els_mto = filter_xarray_by_timestamps(els_mto, start_time, end_time)

### Radiosoundings

In [ ]:
cendrosa_dir='../../../obs/LIAISE/LIAISE_LA-CENDROSA_CNRM_RS_L2'
cen_files_list=[cendrosa_files_1507,cendrosa_files_1607,cendrosa_files_1707,cendrosa_files_2007,cendrosa_files_2107,cendrosa_files_2207,cendrosa_files_2707]
cen_times_list=[cen_1507_times,cen_1607_times,cen_1707_times,cen_2007_times,cen_2107_times,cen_2207_times,cen_2707_times]

els_dir='../../../obs/LIAISE/LIAISE_ELS-PLANS_UKMO_radiosondes_L1'
els_files_list=[els_files_1507,els_files_1607,els_files_1707,els_files_2007,els_files_2107,els_files_2207,els_files_2707]
els_times_list=[els_1507_times,els_1607_times,els_1707_times,els_2007_times,els_2107_times,els_2207_times,els_2707_times]

In [ ]:
cen_RS={}
els_RS={}
for i in range(len(cen_files_list)):
    # print(RS_days_list[i])
    cen_RS_day=RS_extract_full_day(cen_files_list[i], cen_times_list[i], cendrosa_dir, format_Cendrosa_RS)
    els_RS_day=RS_extract_full_day(els_files_list[i], els_times_list[i], els_dir, format_ElsPlans_RS)
    cen_RS[RS_days_list[i]]=cen_RS_day
    els_RS[RS_days_list[i]]=els_RS_day

## MesoNH

In [ ]:
mesoNH_dir = '../../../mesoNH_simulations'

In [ ]:
def format_MesoNH_outputs_basic(filename, ds_name=None):
    ds = xr.open_mfdataset(filename)
    ds.attrs['name']= ds_name

    rename_dict={
        'time':'start_time',
        'longitude':'lon',
        'latitude':'lat',
        'HBLTOP':'s_pblh',
    }
    ds = ds.rename(rename_dict)

    # Add time coordinate
    # Assuming 'time' coordinate is the starting time.
    base_time = pd.to_datetime(ds["start_time"].isel(start_time=0).item())
    print(f"Base time for the dataset: {base_time}")
    # Create a timedelta for each hour
    hourly_offsets = pd.to_timedelta(ds["record"].values, unit='h')
    new_time_vals = base_time + hourly_offsets
    # Assign new_time as a new coordinate, linked to the 'record' dimension
    ds = ds.assign_coords(time=("record", new_time_vals))
    ds = ds.swap_dims({"record": "time"})
    #drop start_time and record
    ds = ds.drop_vars(['start_time'])
    ds = ds.assign_coords(time_decimal=ds.time.dt.hour + ds.time.dt.minute / 60)

    if 'LE_ISBA' in ds:
        ds = ds.rename({'LE_ISBA': 'flat'})
        ds['flat'].attrs['long_name'] = 'Latent heat flux'
        ds['flat'].attrs['units'] = 'W m⁻²'
    if 'H_ISBA' in ds:
        ds = ds.rename({'H_ISBA': 'sens'})
        ds['sens'].attrs['long_name'] = 'Sensible heat flux'
        ds['sens'].attrs['units'] = 'W m⁻²'
    if 'HU2M_ISBA' in ds:
        ds['rh2m'] = ds['HU2M_ISBA'] * 100.0 
        ds['rh2m'].attrs['long_name'] = 'Relative humidity at 2m'
        ds['rh2m'].attrs['units'] = '%'
    if 'T2M_ISBA' in ds:
        ds = ds.rename({'T2M_ISBA': 't2m'})
        ds['t2m'] = ds['t2m'] - 273.15
        ds['t2m'].attrs['units'] = '°C'
        ds['t2m'].attrs['long_name'] = 'Temperature at 2m'
    if 'SWD' in ds:
        #remove dimension 'start_time' from SWD
        ds['SWD'] = ds['SWD'].squeeze('start_time', drop=True)
        ds = ds.rename({'SWD': 'SWdnSFC'})
        ds['SWdnSFC'].attrs['long_name'] = 'Downward shortwave radiation at surface'
        ds['SWdnSFC'].attrs['name'] = 'Surface SW down'
        ds['SWdnSFC'].attrs['units'] = 'W m⁻²'
    if 'LWD' in ds:
        #remove dimension 'start_time' from LWD
        ds['LWD'] = ds['LWD'].squeeze('start_time', drop=True)
        ds = ds.rename({'LWD': 'LWdnSFC'})
        ds['LWdnSFC'].attrs['long_name'] = 'Downward longwave radiation at surface'
        ds['LWdnSFC'].attrs['name'] = 'Surface LW down'
        ds['LWdnSFC'].attrs['units'] = 'W m⁻²'
    if 'LWU' in ds:
        #remove dimension 'start_time' from LWU
        ds['LWU'] = ds['LWU'].squeeze('start_time', drop=True)
    if 'SWU' in ds:
        #remove dimension 'start_time' from SWU
        ds['SWU'] = ds['SWU'].squeeze('start_time', drop=True)
    if 'HBLTOP' in ds:
        #remove dimension 'start_time' from HBLTOP
        ds['HBLTOP'] = ds['HBLTOP'].squeeze('start_time', drop=True)

    # remove ds history attribute
    if 'history' in ds.attrs:
        del ds.attrs['history']

    return(ds)
filename='{}/LIAIS.1*.nc'.format(mesoNH_dir)
# filename='{}/LIAIS.1.UT10.nc'.format(mesoNH_dir)
mesoNH = format_MesoNH_outputs_basic(filename)
# mesoNH

In [ ]:
def format_MesoNH_outputs_winds(filename, ds_name=None):
    ds = xr.open_mfdataset(filename)
    ds.attrs['name']= ds_name

    rename_dict={
        'time':'start_time',
        'longitude':'lon',
        'latitude':'lat',
    }
    ds = ds.rename(rename_dict)

    # Add time coordinate
    # Assuming 'time' coordinate is the starting time.
    base_time = pd.to_datetime(ds["start_time"].isel(start_time=0).item())
    print(f"Base time for the dataset: {base_time}")
    # Create a timedelta for each hour
    hourly_offsets = pd.to_timedelta(ds["record"].values, unit='h')
    new_time_vals = base_time + hourly_offsets
    # Assign new_time as a new coordinate, linked to the 'record' dimension
    ds = ds.assign_coords(time=("record", new_time_vals))
    ds = ds.swap_dims({"record": "time"})
    #drop start_time and record
    ds = ds.drop_vars(['start_time'])
    ds = ds.assign_coords(time_decimal=ds.time.dt.hour + ds.time.dt.minute / 60)

    if 'UT' in ds:
        ds['UT'].attrs['long_name'] = '10m U wind component'
        ds['UT'].attrs['units'] = 'm s⁻¹'
        ds['UT'] = ds['UT'].squeeze('start_time', drop=True)
        ds['UT'] =ds['UT'].squeeze('level', drop=True)

    if 'VT' in ds:
        ds['VT'].attrs['long_name'] = '10m V wind component'
        ds['VT'].attrs['units'] = 'm s⁻¹'
        ds['VT'] = ds['VT'].squeeze('start_time', drop=True)
        ds['VT'] =ds['VT'].squeeze('level', drop=True)

    ds = flux_pt_to_mass_pt(ds, basic_vars=['UT', 'VT'])

    #rename UT and VT to u10m and v10m
    if 'UT' in ds:
        ds = ds.rename({'UT': 'u10m'})
        ds['u10m'].attrs['long_name'] = '10m U wind component'
        ds['u10m'].attrs['units'] = 'm s⁻¹'
    if 'VT' in ds:
        ds = ds.rename({'VT': 'v10m'})
        ds['v10m'].attrs['long_name'] = '10m V wind component'
        ds['v10m'].attrs['units'] = 'm s⁻¹'

    # remove ds history attribute
    if 'history' in ds.attrs:
        del ds.attrs['history']
    
    #drop longitude and latitude
    ds = ds.drop_vars(['lon', 'lat'])

    return(ds)

filename='{}/winds/LIAIS.1*.nc'.format(mesoNH_dir)
# filename='{}/LIAIS.1.UT10.nc'.format(mesoNH_dir)
mesoNH_winds = format_MesoNH_outputs_winds(filename)

#add surface winds to mesoNH
mesoNH = xr.merge([mesoNH, mesoNH_winds], compat='override')
mesoNH = add_wind_10m( mesoNH)
mesoNH

In [ ]:
load_4D=True
def format_MesoNH_outputs_4D(filename, ds_name=None):
    ds = xr.open_mfdataset(filename)
    ds.attrs['name']= ds_name


    rename_dict={
        'time':'start_time',
        'longitude':'lon',
        'latitude':'lat',
        'THT':'theta',
        'TKET':'tke',
        'UT':'vitu',
        'VT':'vitv',
        'WT':'vitw',
    }
    ds = ds.rename(rename_dict)

    # Add time coordinate
    # Assuming 'time' coordinate is the starting time.
    base_time = pd.to_datetime(ds["start_time"].isel(start_time=0).item())
    print(f"Base time for the dataset: {base_time}")
    #Create a timedelta for each hour
    hourly_offsets = pd.to_timedelta(ds["record"].values, unit='h')
    new_time_vals = base_time + hourly_offsets
    # Assign new_time as a new coordinate, linked to the 'record' dimension
    ds = ds.assign_coords(time=("record", new_time_vals))
    ds = ds.swap_dims({"record": "time"})
    #drop start_time
    # ds = ds.drop_vars(['start_time'])
    # ds = ds.drop_dims(['start_time'])
    ds = ds.squeeze('start_time', drop=True)

    ds = add_specific_humidity_from_mixing_ratio(ds)

    ds['altitude_agl'] = ds['level']

    ds = ds.assign_coords(time_decimal=ds.time.dt.hour + ds.time.dt.minute / 60)

    #remove ds history attribute
    if 'history' in ds.attrs:
        del ds.attrs['history']

    ds = flux_pt_to_mass_pt(ds, basic_vars=['vitu', 'vitv', 'vitw'])
    ds = add_wind_speed(ds)
    ds = add_wind_direction(ds)
    
    return ds

# if load_4D:
# filename='{}/4D/LIAIS.1.SEG16.012subset_abl_6var.nc'.format(mesoNH_dir)
filename='{}/4D/LIAIS.1.subset_abl_8vars.nc'.format(mesoNH_dir)

mesoNH_4D = format_MesoNH_outputs_4D(filename)
mesoNH_4D

In [ ]:
#extract pressure at surface from mesoNH_4D and add it to mesoNH
mesoNH = extract_pressure_surface(mesoNH_4D, mesoNH)
#add specific humidity at 2m to mesoNH
mesoNH = add_specific_humidity_2m(mesoNH, t2m_in_kelvin=False, psol_Pa=True)
mesoNH

## Subdataset for ICOLMDZ grid cells and obs sites

### Cell mean

In [ ]:
mesoNH_elsplans_cell= subset_dataset_by_polygon(mesoNH, elsplans_cell, 
                                            name='mesoNH_mean',
                                            plot_color='orange')
mesoNH_cendrosa_cell= subset_dataset_by_polygon(mesoNH, cendrosa_cell, 
                                            name='mesoNH_mean',
                                            plot_color='orange')

In [ ]:
mesoNH_4D_cendrosa_cell = subset_dataset_by_polygon(mesoNH_4D, cendrosa_cell, 
                                            name='mesoNH_mean',
                                            plot_color='orange')

mesoNH_4D_elsplans_cell = subset_dataset_by_polygon(mesoNH_4D, elsplans_cell, 
                                            name='mesoNH_mean',
                                            plot_color='orange')

In [ ]:
mesoNH_4D_cendrosa_cell_mean = mesoNH_4D_cendrosa_cell.mean(dim=['ni','nj'])
mesoNH_4D_cendrosa_cell_mean.attrs = mesoNH_4D_cendrosa_cell.attrs

mesoNH_4D_elsplans_cell_mean = mesoNH_4D_elsplans_cell.mean(dim=['ni','nj'])
mesoNH_4D_elsplans_cell_mean.attrs = mesoNH_4D_elsplans_cell.attrs

In [ ]:
#quantiles for mesoNH
mesoNH_4D_cendrosa_cell_q25 = mesoNH_4D_cendrosa_cell.quantile(0.25, dim=['ni','nj'])
mesoNH_4D_cendrosa_cell_q75 = mesoNH_4D_cendrosa_cell.quantile(0.75, dim=['ni','nj'])
mesoNH_4D_elsplans_cell_q25 = mesoNH_4D_elsplans_cell.quantile(0.25, dim=['ni','nj'])
mesoNH_4D_elsplans_cell_q75 = mesoNH_4D_elsplans_cell.quantile(0.75, dim=['ni','nj'])

### Exact cell

In [ ]:
#select subdataset for precisely each site
mesoNH_cendrosa_exact = select_dataset_lon_lat(mesoNH, 
                                        lon=Cendrosa_Longitude, lat=Cendrosa_Latitude, 
                                        name='mesoNH_exact',
                                        plot_color='purple')

In [ ]:
mesoNH_elsplans_exact = select_dataset_lon_lat(mesoNH, 
                                        lon=ElsPlans_Longitude, lat=ElsPlans_Latitude, 
                                        name='mesoNH_exact', 
                                        plot_color='purple')

In [ ]:
mesoNH_4D_cendrosa_exact = select_dataset_lon_lat(mesoNH_4D,
                                        lon=Cendrosa_Longitude, lat=Cendrosa_Latitude,
                                        name='mesoNH_exact',
                                        plot_color='purple')

In [ ]:
mesoNH_4D_elsplans_exact = select_dataset_lon_lat(mesoNH_4D,
                                        lon=ElsPlans_Longitude, lat=ElsPlans_Latitude,
                                        name='mesoNH_exact',
                                        plot_color='purple')

### Subset for a given class of flat or fsens

In [ ]:
def subset_bin(ds, var, timestamp, bin_min, bin_max):
    """
    Subset an xarray Dataset based on a variable's value range at a specific timestamp.

    Parameters:
    ds (xarray.Dataset): The input dataset.
    var (str): The variable name to filter on.
    timestamp (str or pd.Timestamp): The specific timestamp to consider.
    bin_min (float): The minimum value of the bin (inclusive).
    bin_max (float): The maximum value of the bin (exclusive).

    Returns:
    xarray.Dataset: A subset of the original dataset where the variable's value at the specified timestamp falls within the given range.
    """

    # Select data at the specified timestamp
    ds_at_time = ds.sel(time=timestamp, method='nearest')

    # Create a boolean mask for the variable within the specified range
    mask = (ds_at_time[var] >= bin_min) & (ds_at_time[var] < bin_max)

    # Apply the mask to the original dataset
    subset_ds = ds_at_time.where(ds_at_time[var] < bin_max).where(ds_at_time[var] >= bin_min)
    subset_mean = subset_ds.mean(dim=['ni','nj'])
    subset_q25 = subset_ds.quantile(0.25, dim=['ni','nj'])
    subset_q75 = subset_ds.quantile(0.75, dim=['ni','nj'])

    return subset_ds, subset_mean, subset_q25, subset_q75

In [ ]:
def subset_with_mask(ds_4D: xr.Dataset, ds_mask: xr.Dataset, var: str, timestamp: str or pd.Timestamp, bin_min: float, bin_max: float):
    """
    Subsets one xarray Dataset (ds_4D) by applying a mask computed from another (ds_mask).

    The mask is determined by a variable's value range in ds_mask at a specific timestamp.
    The resulting subset_ds_4D will contain data only for grid cells where the variable
    in ds_mask falls within the specified bin limits.

    Parameters:
    ds_4D (xarray.Dataset): The dataset to be subsetted. This is the 4D dataset.
    ds_mask (xarray.Dataset): The dataset used to compute the mask.
    var (str): The variable name to filter on.
    timestamp (str or pd.Timestamp): The specific timestamp to consider for the mask.
    bin_min (float): The minimum value of the bin (inclusive).
    bin_max (float): The maximum value of the bin (exclusive).

    Returns:
    tuple: A tuple containing:
           - subset_ds (xarray.Dataset): The subsetted dataset.
           - subset_mean (xarray.DataArray): The mean of the subsetted data across spatial dimensions.
           - subset_q25 (xarray.DataArray): The 25th percentile of the subsetted data.
           - subset_q75 (xarray.DataArray): The 75th percentile of the subsetted data.
    """

    # 1. Create a 2D mask from ds_mask at the specified timestamp
    # We select the timestamp and then apply the bin limits
    ds_mask_at_time = ds_mask.sel(time=timestamp, method='nearest')
    mask = (ds_mask_at_time[var] >= bin_min) & (ds_mask_at_time[var] < bin_max)

    # 2. Apply the 2D mask to the 4D dataset.
    # xarray automatically broadcasts the 2D mask across the time dimension.
    ds4D_at_time = ds_4D.sel(time=timestamp, method='nearest')
    subset_ds = ds4D_at_time.where(mask)

    # 3. Compute statistics on the subsetted dataset
    subset_mean = subset_ds.mean(dim=['ni','nj'])
    subset_q25 = subset_ds.quantile(0.25, dim=['ni','nj'])
    subset_q75 = subset_ds.quantile(0.75, dim=['ni','nj'])

    return subset_ds, subset_mean, subset_q25, subset_q75

In [ ]:
cendrosa_1507_flat_bin=(300,325)
cendrosa_1507_sens_bin=(175,200)
elsplans_1507_flat_bin=(0,25)
elsplans_1507_sens_bin=(475,500)

cendrosa_2007_flat_bin=(350,375)
cendrosa_2007_sens_bin=(125,150)
elsplans_2007_flat_bin=(0,25)
elsplans_2007_sens_bin=(475,500)

In [ ]:
adjust_bin_value=0
var = 'sens'
#Cendrosa
ds_mask=mesoNH_cendrosa_cell
ds_4D=mesoNH_4D_cendrosa_cell

timestamp = '2021-07-15T12:00:00'
bin_min = cendrosa_1507_sens_bin[0]-adjust_bin_value
bin_max = cendrosa_1507_sens_bin[1]+adjust_bin_value
mesoNH_cendrosa_1507_sens, mesoNH_cendrosa_1507_sens_mean, mesoNH_cendrosa_1507_sens_q25, mesoNH_cendrosa_1507_sens_q75 = subset_with_mask(ds_4D, ds_mask, var, timestamp, bin_min, bin_max)

timestamp = '2021-07-20T12:00:00'
bin_min = cendrosa_2007_sens_bin[0]-adjust_bin_value
bin_max = cendrosa_2007_sens_bin[1]+adjust_bin_value
mesoNH_cendrosa_2007_sens, mesoNH_cendrosa_2007_sens_mean, mesoNH_cendrosa_2007_sens_q25, mesoNH_cendrosa_2007_sens_q75 = subset_with_mask(ds_4D, ds_mask, var, timestamp, bin_min, bin_max)

#Els Plans
ds_mask=mesoNH_elsplans_cell
ds_4D=mesoNH_4D_elsplans_cell

timestamp = '2021-07-15T12:00:00'
bin_min = elsplans_1507_sens_bin[0]-50
bin_max = elsplans_1507_sens_bin[1]+adjust_bin_value
mesoNH_elsplans_1507_sens, mesoNH_elsplans_1507_sens_mean, mesoNH_elsplans_1507_sens_q25, mesoNH_elsplans_1507_sens_q75 = subset_with_mask(ds_4D, ds_mask, var, timestamp, bin_min, bin_max)

timestamp = '2021-07-20T12:00:00'
bin_min = elsplans_2007_sens_bin[0]-adjust_bin_value
bin_max = elsplans_2007_sens_bin[1]+adjust_bin_value
mesoNH_elsplans_2007_sens, mesoNH_elsplans_2007_sens_mean, mesoNH_elsplans_2007_sens_q25, mesoNH_elsplans_2007_sens_q75 = subset_with_mask(ds_4D, ds_mask, var, timestamp, bin_min, bin_max)

In [ ]:
adjust_bin_value=0
var = 'flat'
#Cendrosa
ds_mask=mesoNH_cendrosa_cell
ds_4D=mesoNH_4D_cendrosa_cell

timestamp = '2021-07-15T12:00:00'
bin_min = cendrosa_1507_flat_bin[0]-adjust_bin_value
bin_max = cendrosa_1507_flat_bin[1]+adjust_bin_value
mesoNH_cendrosa_1507_flat, mesoNH_cendrosa_1507_flat_mean, mesoNH_cendrosa_1507_flat_q25, mesoNH_cendrosa_1507_flat_q75 = subset_with_mask(ds_4D, ds_mask, var, timestamp, bin_min, bin_max)

timestamp = '2021-07-20T12:00:00'
bin_min = cendrosa_2007_flat_bin[0]-adjust_bin_value
bin_max = cendrosa_2007_flat_bin[1]+adjust_bin_value
mesoNH_cendrosa_2007_flat, mesoNH_cendrosa_2007_flat_mean, mesoNH_cendrosa_2007_flat_q25, mesoNH_cendrosa_2007_flat_q75 = subset_with_mask(ds_4D, ds_mask, var, timestamp, bin_min, bin_max)

#Els Plans
ds_mask=mesoNH_elsplans_cell
ds_4D=mesoNH_4D_elsplans_cell

timestamp = '2021-07-15T12:00:00'
bin_min = elsplans_1507_flat_bin[0]+adjust_bin_value
bin_max = elsplans_1507_flat_bin[1]+adjust_bin_value
mesoNH_elsplans_1507_flat, mesoNH_elsplans_1507_flat_mean, mesoNH_elsplans_1507_flat_q25, mesoNH_elsplans_1507_flat_q75 = subset_with_mask(ds_4D, ds_mask, var, timestamp, bin_min, bin_max)

timestamp = '2021-07-20T12:00:00'
bin_min = elsplans_2007_flat_bin[0]-adjust_bin_value
bin_max = elsplans_2007_flat_bin[1]+adjust_bin_value
mesoNH_elsplans_2007_flat, mesoNH_elsplans_2007_flat_mean, mesoNH_elsplans_2007_flat_q25, mesoNH_elsplans_2007_flat_q75 = subset_with_mask(ds_4D, ds_mask, var, timestamp, bin_min, bin_max)

## Load LMDZ files

In [ ]:
nat_dir='../../../JZ_simu_outputs/LAM/LAM_1500_60/halfhour_native'

boost_irrig_dir= '../../../JZ_simu_outputs/LAM/LAM_1500_60/boost_irrig'

In [ ]:
noirr=True
irr100=True
routing_restart_full_beta1=True
nudged_lmdz=False

In [ ]:
#load LMDZ
if noirr:
    lmdz_noirr_file = '{}/sim267_20210701_20210731_HF_histhf.nc'.format(nat_dir)
    lmdz_noirr = format_lmdz_HF(lmdz_noirr_file, 'red', 'noirr')
    lmdz_noirr = lmdz_noirr.sel(time=slice('2021-07-14', '2021-07-30'))
    cen_lmdz_noirr, els_lmdz_noirr  , sop_cen_lmdz_noirr,   sop_els_lmdz_noirr = select_liaise_sites_sop(lmdz_noirr, 'no_irr')


In [ ]:
if irr100:
    dir=boost_irrig_dir
    sim_name='sim275'
    color='blue'
    label='irr'

    lmdz_irr100_file = '{}/{}_20210701_20210731_HF_histhf.nc'.format(dir, sim_name)
    lmdz_irr100 = format_lmdz_HF(lmdz_irr100_file, color, label)
    lmdz_irr100 = lmdz_irr100.sel(time=slice('2021-07-14', '2021-07-30'))
    cen_lmdz_irr100, els_lmdz_irr100  , sop_cen_lmdz_irr100,   sop_els_lmdz_irr100 = select_liaise_sites_sop(lmdz_irr100, label)


In [ ]:
if routing_restart_full_beta1 :
    dir=boost_irrig_dir
    sim_name='sim293'
    color='green'
    label='irr_boost'

    lmdz_routingfull_beta1_file = '{}/{}_20210701_20210731_HF_histhf.nc'.format(dir, sim_name)
    lmdz_routingfull_beta1 = format_lmdz_HF(lmdz_routingfull_beta1_file, color, label)
    lmdz_routingfull_beta1 = lmdz_routingfull_beta1.sel(time=slice('2021-07-14', '2021-07-30'))
    cen_lmdz_routingfull_beta1, els_lmdz_routingfull_beta1, sop_cen_lmdz_routingfull_beta1, sop_els_lmdz_routingfull_beta1 = select_liaise_sites_sop(lmdz_routingfull_beta1, label)

In [ ]:
if routing_restart_full_beta1 and nudged_lmdz:
    dir=boost_irrig_dir
    sim_name='sim299'
    color='orange'
    label='irr_boost_nudged'

    lmdz_routingfull_beta1_nudged_file = '{}/{}_20210701_20210731_HF_histhf.nc'.format(dir, sim_name)
    lmdz_routingfull_beta1_nudged = format_lmdz_HF(lmdz_routingfull_beta1_nudged_file, color, label)
    lmdz_routingfull_beta1_nudged = lmdz_routingfull_beta1_nudged.sel(time=slice('2021-07-14', '2021-07-30'))
    cen_lmdz_routingfull_beta1_nudged, els_lmdz_routingfull_beta1_nudged, sop_cen_lmdz_routingfull_beta1_nudged, sop_els_lmdz_routingfull_beta1_nudged = select_liaise_sites_sop(lmdz_routingfull_beta1_nudged, label)

## Load ORC files

In [ ]:
load_ORC=False
def format_ORC_HF(filename, color='blue', ds_name=None):
    ds = xr.open_mfdataset(filename)
    ds.attrs['name']= ds_name

    rename_dict={
        # 'time':'start_time',
        # 'longitude':'lon',
        # 'latitude':'lat',
    }
    ds = ds.rename(rename_dict)

if load_ORC:
    #noirr
    orc_noirr_file =  '{}/sim267_20210701_20210731_HF_sechiba_out_2.nc'.format(nat_dir)
    orc_noirr = format_ORC_HF(orc_noirr_file, color='red', ds_name='no_irr')
    orc_noirr = orc_noirr.sel(time=slice('2021-07-14', '2021-07-30'))
    cen_orc_noirr, els_orc_noirr, sop_cen_orc_noirr, sop_els_orc_noirr = select_liaise_sites_sop(orc_noirr, 'no_irr')

    #irr100
    orc_irr100_file = '{}/sim275_20210701_20210731_HF_sechiba_out_2.nc'.format(boost_irrig_dir)
    orc_irr100 = format_ORC_HF(orc_irr100_file, color='blue', ds_name='irr')
    orc_irr100 = orc_irr100.sel(time=slice('2021-07-14', '2021-07-30'))
    cen_orc_irr100, els_orc_irr100, sop_cen_orc_irr100, sop_els_orc_irr100 = select_liaise_sites_sop(orc_irr100, 'irr')

    #routing_restart_full_beta1
    orc_routingfull_beta1_file = '{}/sim293_20210701_20210731_HF_sechiba_out_2.nc'.format(boost_irrig_dir)
    orc_routingfull_beta1 = format_ORC_HF(orc_routingfull_beta1_file, color='green', ds_name='irr_boost')
    orc_routingfull_beta1 = orc_routingfull_beta1.sel(time=slice('2021-07-14', '2021-07-30'))
    cen_orc_routingfull_beta1, els_orc_routingfull_beta1, sop_cen_orc_routingfull_beta1, sop_els_orc_routingfull_beta1 = select_liaise_sites_sop(orc_routingfull_beta1, 'irr_boost')        

## Subset LMDZ and MesoNH ds for LIAISE sites and IOP days

In [ ]:
#SOP
day_start_times=[day1_start_time, day2_start_time, day3_start_time, day4_start_time, day5_start_time, day6_start_time, day7_start_time]
day_end_times  =[day1_end_time, day2_end_time, day3_end_time, day4_end_time, day5_end_time, day6_end_time, day7_end_time]

In [ ]:
iop_cen_lmdz_noirr = {}
iop_cen_lmdz_irr100 = {}
iop_cen_lmdz_routingfull_beta1 = {}
iop_cen_lmdz_routingfull_beta1_nudged = {}
iop_els_lmdz_noirr = {}
iop_els_lmdz_irr100 = {}
iop_els_lmdz_routingfull_beta1 = {}
iop_els_lmdz_routingfull_beta1_nudged = {}
for i,day in enumerate(RS_days_list):
    iop_cen_lmdz_noirr[day] = filter_xarray_by_timestamps(cen_lmdz_noirr, day_start_times[i], day_end_times[i])
    iop_cen_lmdz_irr100[day] = filter_xarray_by_timestamps(cen_lmdz_irr100, day_start_times[i], day_end_times[i])
    iop_cen_lmdz_routingfull_beta1[day] = filter_xarray_by_timestamps(cen_lmdz_routingfull_beta1, day_start_times[i], day_end_times[i])
    iop_els_lmdz_noirr[day] = filter_xarray_by_timestamps(els_lmdz_noirr, day_start_times[i], day_end_times[i])
    iop_els_lmdz_irr100[day] = filter_xarray_by_timestamps(els_lmdz_irr100, day_start_times[i], day_end_times[i])
    iop_els_lmdz_routingfull_beta1[day] = filter_xarray_by_timestamps(els_lmdz_routingfull_beta1, day_start_times[i], day_end_times[i])

    if nudged_lmdz:
        iop_cen_lmdz_routingfull_beta1_nudged[day] = filter_xarray_by_timestamps(cen_lmdz_routingfull_beta1_nudged, day_start_times[i], day_end_times[i])
        iop_els_lmdz_routingfull_beta1_nudged[day] = filter_xarray_by_timestamps(els_lmdz_routingfull_beta1_nudged, day_start_times[i], day_end_times[i])

In [ ]:
iop_cen_mesoNH_4D_exact = {}
iop_els_mesoNH_4D_exact = {}
iop_cen_mesoNH_4D_cell_mean = {}
iop_els_mesoNH_4D_cell_mean = {}
#quantiles
iop_cen_mesoNH_4D_cell_q25 = {}
iop_cen_mesoNH_4D_cell_q75 = {}
iop_els_mesoNH_4D_cell_q25 = {}
iop_els_mesoNH_4D_cell_q75 = {}
for i,day in enumerate(RS_days_list):
    iop_cen_mesoNH_4D_exact[day] = filter_xarray_by_timestamps(mesoNH_4D_cendrosa_exact, day_start_times[i], day_end_times[i])
    iop_els_mesoNH_4D_exact[day] = filter_xarray_by_timestamps(mesoNH_4D_elsplans_exact, day_start_times[i], day_end_times[i])
    iop_cen_mesoNH_4D_cell_mean[day] = filter_xarray_by_timestamps(mesoNH_4D_cendrosa_cell_mean, day_start_times[i], day_end_times[i])
    iop_els_mesoNH_4D_cell_mean[day] = filter_xarray_by_timestamps(mesoNH_4D_elsplans_cell_mean, day_start_times[i], day_end_times[i])
    #quantiles
    iop_cen_mesoNH_4D_cell_q25[day] = filter_xarray_by_timestamps(mesoNH_4D_cendrosa_cell_q25, day_start_times[i], day_end_times[i])
    iop_cen_mesoNH_4D_cell_q75[day] = filter_xarray_by_timestamps(mesoNH_4D_cendrosa_cell_q75, day_start_times[i], day_end_times[i])
    iop_els_mesoNH_4D_cell_q25[day] = filter_xarray_by_timestamps(mesoNH_4D_elsplans_cell_q25, day_start_times[i], day_end_times[i])
    iop_els_mesoNH_4D_cell_q75[day] = filter_xarray_by_timestamps(mesoNH_4D_elsplans_cell_q75, day_start_times[i], day_end_times[i])

# Tests

In [ ]:
stop

## Maps

In [ ]:
ds=mesoNH
var=''

mean_ds = ds[var].mean(dim='time')
# mean_ds = mean_ds[0,:,:]  # Select the specific level
nice_map_mesoNH(mean_ds, poly=both_cells)

In [ ]:
var='flat'
vmin=0
vmax=500
# vmin,vmax=None, None
cmap=blues
map_mesoNH_timestamp_restrict(mesoNH, 
                     var,
                     vmin=vmin, vmax=vmax, cmap=cmap,
                     timestamp='2021-07-20T12:00:00',
                     add_liaise=True,
                    lon_min=0.6, 
                    lon_max=1.4, 
                    lat_min=41.37, 
                    lat_max=41.92,
                    poly=both_cells
                     )

In [ ]:
#list of all variables names in mesoNH
# mesoNH.data_vars.keys()
varlist=['HBLTOP', 'sens', 'flat', 'rh2m', 't2m',  'LWD', 'LWU', 'SWdnSFC',
          'SWI1_ISBA', 'SWI3_ISBA', 'TG1_ISBA', 'TG3_ISBA']

In [ ]:
for var in varlist:
    vmin=0
    vmax=500
    vmin,vmax=None, None
    cmap=blues
    map_mesoNH_timestamp_restrict(mesoNH, 
                        var,
                        vmin=vmin, vmax=vmax, cmap=cmap,
                        timestamp='2021-07-20T12:00:00',
                        add_liaise=True,
                        # lon_min=0.6, 
                        # lon_max=1.4, 
                        # lat_min=41.37, 
                        # lat_max=41.92,
                        # poly=both_cells
                        )

In [ ]:
var='flat'
vmin=-0
vmax=200
# vmin,vmax=None, None
cmap=wet8
map_mesoNH_mean_restrict(mesoNH, 
                 var,
                 vmin=vmin, vmax=vmax, cmap=cmap,
                #  title='ISBA latent heat flux mean (14-31 July)',
                title='off',
                 add_liaise=True,
                lon_min=0.55, 
                lon_max=1.4, 
                lat_min=41.37, 
                lat_max=41.9,
                poly=both_cells,
                label='Latent heat flux (W m⁻²)',
                 )

In [ ]:
map_mesoNH_timestamp(mesoNH_elsplans_cell,
                 var='flat',
                 vmin=0, vmax=250, cmap=myvir,
                #  title='Latent Heat Flux mean (14-31/07/2021) in Ebro basin',
                 add_liaise=True,
                 timestamp='2021-07-20T12:00:00',
                 )

## Means and bins

In [ ]:
ds=mesoNH_cendrosa_cell
# ds=mesoNH_elsplans_cell
timestamp='2021-07-20T12:00:00'
timestamp='2021-07-15T12:00:00'
var='sens'
xmin=0
xmax=550
ylim=(0,52)
nbins=22
ds_list=[mesoNH_cendrosa_exact, cen_lmdz_noirr, cen_lmdz_irr100, cen_lmdz_routingfull_beta1, cen_mto]
bins_timestamp(ds, var, timestamp, nbins=nbins, xmin=xmin, xmax=xmax, ylim=ylim, ds_list=ds_list, site='- La Cendrosa')

ds=mesoNH_elsplans_cell
ds_list=[mesoNH_elsplans_exact, els_lmdz_noirr, els_lmdz_irr100, els_lmdz_routingfull_beta1, els_mto]
bins_timestamp(ds, var, timestamp, nbins=nbins, xmin=xmin, xmax=xmax, ylim=ylim, ds_list=ds_list, site='- Els Plans')

In [ ]:
ds=mesoNH_cendrosa_cell
ds=mesoNH_elsplans_cell
# timestamp='2021-07-20T12:00:00'
var='flat'
xmin=0
xmax=525
nbins=21
title='Distribution of mean latent heat flux at 12UTC (14-31/07/2021)'
xlabel='Latent heat flux (W m⁻²)'
bins_hour(ds, var, 12, nbins=nbins, xmin=xmin, xmax=xmax, 
          title=title, xlabel=xlabel)

## Time series

In [ ]:
var='wind_speed_10m'
ds=mesoNH
vmin=0
vmax=200
vmin,vmax=None, None
time_series_lonlat_mesoNH(ds, var, 
                          lon=ElsPlans_lon, 
                          lat=ElsPlans_lat,
                          vmin=vmin, vmax=vmax
                          )

In [ ]:
var='sens'
ds=mesoNH_cendrosa_cell
ds=mesoNH_elsplans_cell
vmin=0
vmax=200
vmin,vmax=None, None
time_series_ave_mesoNH(ds, var, 
                    #    vmin=vmin, vmax=vmax
                       )

### TS and DC

In [ ]:
var='sens'
ds1= cen_lmdz_noirr
ds2= cen_lmdz_irr100
ds3= cen_lmdz_routingfull_beta1
ds4=mesoNH_cendrosa_cell
ds5=mesoNH_cendrosa_exact
ds6=restrict_cen_mto
ds7 = cen_lmdz_routingfull_beta1_nudged

# ds1= els_lmdz_noirr
# ds2= els_lmdz_irr100
# ds3= els_lmdz_routingfull_beta1
# ds4=mesoNH_elsplans_cell
# ds5=mesoNH_elsplans_exact
# ds6=restrict_els_mto

ds4.attrs['show_envelope'] = True

# ds1['q2m'].attrs['long_name'] = '2-m specific humidity'

ds_list= [ds6, ds1, ds2, ds3, ds5, ds4]  
ds_list= [ds1, ds2, ds3, ds5, ds4]  
ds_list=[ds4]
ds_list=[ds6,ds3, ds7]

# title=f'{ds1[var].attrs['long_name']} ({ds1[var].attrs['units']})'
# if ds1[var].attrs['units'] == 'W m⁻²':
#     title=f'{ds1[var].attrs["long_name"]} (W m⁻²)'
title='off'

# ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
# ylabel = ds1[var].attrs['name'] + ' (' + ds1[var].attrs['units'] + ')'
ylabel=var + ' (' + ds1[var].attrs['units'] + ')'

In [ ]:
vmin=0
vmax=100
vmin,vmax=None, None

diurnal_cycle_ave(ds_list, var,
                 vmin=vmin, vmax=vmax,
                 title=title,
                 legend_out=None,
                 xlabel='Time (UTC)',
                 ylabel=ylabel,
                 ds_colors=True,
                figsize=(4,4),
                envelope=False
                 )

In [ ]:
stop
vmin=-0
vmax=100
vmin,vmax=None, None

time_series_ave(ds_list, var, 
                vmin=vmin, vmax=vmax,
                title=title,
                legend_out=False,
                xlabel='Date',
                ylabel=ylabel,
                ds_colors=True,
                figsize=(7, 4),
                envelope=False
                )

## Vertical profiles

In [ ]:
var='presnivs'
altvar='altitude_agl'

ds1=cen_lmdz_noirr
ds2=cen_lmdz_irr100
ds3=cen_lmdz_routingfull_beta1
ds4=mesoNH_4D_cendrosa_cell_mean
ds5=mesoNH_4D_cendrosa_exact

ds_list= [ds1, ds2, ds3, ds5, ds4]
ds_list=[ds3]

profile_altitude_local_mean(ds_list, var, alt_var=altvar, obs_ds_list=None, figsize=(6,8), ax=None, title=None, altmin=-0, altmax=2000, nbins=None, substract_gl=True, xmin=None, xmax=None, alpha=1., xlabel=None, ylabel=None)

In [ ]:
profiles_cendrosa_withobs=True
profiles_elsplans_withobs=False

ds1=iop_cen_lmdz_noirr
ds2=iop_cen_lmdz_irr100
ds3=iop_cen_lmdz_routingfull_beta1

ds4=iop_els_lmdz_noirr
ds5=iop_els_lmdz_irr100
ds6=iop_els_lmdz_routingfull_beta1

ds7= iop_cen_mesoNH_4D_cell_mean
ds8= iop_cen_mesoNH_4D_exact
ds9= iop_els_mesoNH_4D_cell_mean
ds10= iop_els_mesoNH_4D_exact

ds11= iop_cen_mesoNH_4D_cell_q25
ds12= iop_cen_mesoNH_4D_cell_q75
ds13= iop_els_mesoNH_4D_cell_q25
ds14= iop_els_mesoNH_4D_cell_q75

ds15 = iop_cen_lmdz_routingfull_beta1_nudged
ds16 = iop_els_lmdz_routingfull_beta1_nudged

day='2007'
# times=[5.25,9.25,13.25, 17.25]
times=[12.25]

var, xmin, xmax, xlabel = 'theta', 306, 314, 'Potential temperature (K)'
var, xmin, xmax, xlabel = 'ovap', 2, 14, 'Specific humidity (g kg⁻¹)'
# var, xmin, xmax, xlabel = 'wind_speed', 0, 12, 'Wind speed at 10m (m/s)'
# var, xmin, xmax, xlabel = 'wind_direction', 0, 360, 'Wind direction at 10m (°)'

altmax=3000
# title='Potential temperature (K)'

if profiles_cendrosa_withobs:
    obs_dict=cen_RS[day]
    ds_list_lmdz = [ds1[day], ds2[day], ds3[day]]
    ds_list_mesoNH = [ds7[day], ds8[day]]
    quantiles, quantiles_color = [ds11[day], ds12[day]], 'orange'

    ds_list_lmdz = [ds3[day], ds15[day]]
    ds_list_mesoNH=None
    quantiles=[]
    quantiles_color=None
    #make title with var name and day and time
    title=f'{var} ({day} at 12 UTC)'
    profile_altitude_multipletimes_obs(ds_list_lmdz, obs_dict, var, times, 
                                    title=title, 
                                    ds_list_mesoNH=ds_list_mesoNH,
                                    altmax=altmax, xmin=xmin, xmax=xmax, 
                                    # nbins=5,
                                    substract_gl=True,
                                    simfreq='30mn',
                                    xlabel=xlabel,
                                    ylabel=False,
                                    quantiles=quantiles,
                                    quantiles_color=quantiles_color
                                    )

if profiles_elsplans_withobs:
    obs_dict=els_RS[day]
    ds_list_lmdz = [ds4[day], ds5[day], ds6[day]]
    ds_list_mesoNH = [ds9[day], ds10[day]]
    quantiles, quantiles_color = [ds13[day], ds14[day]], 'orange'
    profile_altitude_multipletimes_obs(ds_list_lmdz, obs_dict, var, times, 
                                    # title=title, 
                                    ds_list_mesoNH=ds_list_mesoNH,
                                    altmax=altmax, xmin=xmin, xmax=xmax, 
                                    # nbins=5,
                                    substract_gl=True,
                                    simfreq='30mn',
                                    xlabel=xlabel,
                                    ylabel=False,
                                    quantiles=quantiles,
                                    quantiles_color=quantiles_color
                                    )

In [ ]:
#vars without obs
profiles_cendrosa=True
profiles_elsplans=True

ds1=iop_cen_lmdz_noirr
ds2=iop_cen_lmdz_irr100
ds3=iop_cen_lmdz_routingfull_beta1

ds4=iop_els_lmdz_noirr
ds5=iop_els_lmdz_irr100
ds6=iop_els_lmdz_routingfull_beta1

ds7= iop_cen_mesoNH_4D_cell_mean
ds8= iop_cen_mesoNH_4D_exact
ds9= iop_els_mesoNH_4D_cell_mean
ds10= iop_els_mesoNH_4D_exact

ds11= iop_cen_mesoNH_4D_cell_q25
ds12= iop_cen_mesoNH_4D_cell_q75
ds13= iop_els_mesoNH_4D_cell_q25
ds14= iop_els_mesoNH_4D_cell_q75

day='2007'
# times=[5.25,9.25,13.25, 17.25]
times=[12.25]

var='temp'
# xmin, xmax = None, None
xlabel=None

var, xmin, xmax, xlabel = 'tke', 0, 2.5, 'Turbulent kinetic energy (m² s⁻²)'

altmax=3000
# title='Potential temperature (K)'
if profiles_cendrosa:
    #cendrosa
    ds_list_lmdz = [ds1[day], ds2[day], ds3[day]]
    ds_list_mesoNH = [ds7[day], ds8[day]]
    quantiles, quantiles_color = [ds11[day], ds12[day]], 'orange'
    profile_altitude_multipletimes_mean(ds_list_lmdz, var, times, 
                                    # title=title, 
                                    ds_list_mesoNH=ds_list_mesoNH,
                                    altmax=altmax, xmin=xmin, xmax=xmax, 
                                    # nbins=5,
                                    substract_gl=True,
                                    simfreq='30mn',
                                    # xlabel=xlabel,
                                    # ylabel=False,
                                    quantiles=quantiles,
                                    quantiles_color=quantiles_color
                                    )

if profiles_elsplans:
    #elsplans
    ds_list_lmdz = [ds4[day], ds5[day], ds6[day]]
    ds_list_mesoNH = [ds9[day], ds10[day]]
    quantiles, quantiles_color = [ds13[day], ds14[day]], 'orange'
    profile_altitude_multipletimes_mean(ds_list_lmdz, var, times, 
                                        # title=title, 
                                        altmax=altmax, xmin=xmin, xmax=xmax, 
                                        # nbins=5,
                                        substract_gl=True,
                                        simfreq='30mn',
                                        # xlabel=xlabel,
                                        # ylabel=False,
                                        ds_list_mesoNH=ds_list_mesoNH,
                                        quantiles=quantiles,
                                        quantiles_color=quantiles_color
                                        )

# Figures

## Map for LIAISE methods

In [ ]:
savefig=Truevar='flat'
vmin=-0
vmax=200
# vmin,vmax=None, None
cmap=wet8
map_mesoNH_mean_restrict(mesoNH, 
                 var,
                 vmin=vmin, vmax=vmax, cmap=cmap,
                #  title='ISBA latent heat flux mean (14-31 July)',
                title='off',
                 add_liaise=True,
                lon_min=0.55, 
                lon_max=1.4, 
                lat_min=41.37, 
                lat_max=41.9,
                poly=both_cells,
                label='Latent heat flux (W m⁻²)',
                 )
if savefig:
    plt.savefig('figures/chap5/liaise_sites_mean_mesoNH.png', dpi=300, bbox_inches='tight')

## Surface variables on both sites, time series and mean diurnal cycle (15-31/07)

### La Cendrosa

In [ ]:
savefig=True
plot_TS=False
plot_DC=True

varlist=['flat', 'sens', 't2m', 'q2m']
varlist=['wind_speed_10m', 'wind_direction_10m']
# varlist=['SWdnSFC', 'LWdnSFC']

ds1= cen_lmdz_noirr
ds2= cen_lmdz_irr100
ds3= cen_lmdz_routingfull_beta1
ds4=mesoNH_cendrosa_cell
ds4.attrs['show_envelope'] = True
ds5=mesoNH_cendrosa_exact
ds6=restrict_cen_mto
ds_list= [ds6, ds1, ds2, ds3, ds5, ds4]  

title='off'

#TS
mins=[0,-150,15,0]
maxs=[700,450,35,0.015]
for var in varlist:
    if not plot_TS:
        break
    ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
    vmin=mins[varlist.index(var)]
    vmax=maxs[varlist.index(var)]
    vmin,vmax=None, None
    time_series_ave(ds_list, var, 
                    vmin=vmin, vmax=vmax,
                    title=title,
                    legend_out=None,
                    xlabel='Date',
                    ylabel=ylabel,
                    ds_colors=True,
                    figsize=(7, 4),
                    envelope=False
                    )
    if savefig:
        plt.savefig(f'figures/chap5/SOP_TS_DC/time_series_cendrosa_{var}.png', dpi=300)

#DC
mins=[0,-150,15,0]
maxs=[520,420,35,0.015]
mins=[0,0]
maxs=[8,360]
for var in varlist:
    if not plot_DC:
        break
    ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
    vmin=mins[varlist.index(var)]
    vmax=maxs[varlist.index(var)]
    # vmin,vmax=None, None
    diurnal_cycle_ave(ds_list, var,
                     vmin=vmin, vmax=vmax,
                     title=title,
                     legend_out=True,
                    #  xlabel='Time (UTC)',
                     ylabel=ylabel,
                     ds_colors=True,
                    figsize=(4,4),
                    envelope=True
                     )
    if savefig:
        plt.savefig(f'figures/chap5/SOP_TS_DC/diurnal_cycle_cendrosa_{var}.png', dpi=300, bbox_inches='tight')

### Els Plans

In [ ]:
savefig=True
plot_TS=False
plot_DC=True

varlist=['flat', 'sens', 't2m', 'q2m']
varlist=['wind_speed_10m', 'wind_direction_10m']
# varlist=['SWdnSFC', 'LWdnSFC']

# varlist=stop


ds1= els_lmdz_noirr
ds2= els_lmdz_irr100
ds3= els_lmdz_routingfull_beta1
ds4=mesoNH_elsplans_cell
ds4.attrs['show_envelope'] = True
ds5=mesoNH_elsplans_exact
ds6=restrict_els_mto
ds_list= [ds6, ds1, ds2, ds3, ds5, ds4]

title='off'
#TS
# mins=[0,-150,15,0]
# maxs=[700,450,35,0.015]
for var in varlist:
    if not plot_TS:
        break
    ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
    vmin=mins[varlist.index(var)]
    vmax=maxs[varlist.index(var)]
    vmin,vmax=None, None
    time_series_ave(ds_list, var, 
                    # vmin=vmin, vmax=vmax,
                    title=title,
                    legend_out=None,
                    xlabel='Date',
                    ylabel=ylabel,
                    ds_colors=True,
                    figsize=(7, 4),
                    envelope=False
                    )
    if savefig:
        plt.savefig(f'figures/chap5/SOP_TS_DC/time_series_elsplans_{var}.png', dpi=300)
#DC
mins=[0,-150,15,0]
maxs=[520,420,35,0.015]
mins=[0, 0]
maxs=[8, 360]
for var in varlist:
    if not plot_DC:
        break
    ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
    vmin=mins[varlist.index(var)]
    vmax=maxs[varlist.index(var)]
    # vmin,vmax=None, None
    diurnal_cycle_ave(ds_list, var,
                     vmin=vmin, vmax=vmax,
                     title=title,
                     legend_out=True,
                    #  xlabel='Time (UTC)',
                     ylabel=ylabel,
                     ds_colors=True,
                    figsize=(4,4),
                    envelope=True
                     )
    if savefig:
        plt.savefig(f'figures/chap5/SOP_TS_DC/diurnal_cycle_elsplans_{var}.png', dpi=300, bbox_inches='tight')

### Times series to characterize obs days

In [ ]:
savefig=False

varlist=['flat', 'sens', 't2m', 'q2m', 'SWdnSFC', 'LWdnSFC', 'psol','wind_speed_10m', 'wind_direction_10m']
varlist=['precip']

ds1=restrict_cen_mto
ds2=restrict_els_mto
#restrict to period between 15 and 27 July
ds1=ds1.sel(time=slice('2021-07-15', '2021-07-27'))
ds2=ds2.sel(time=slice('2021-07-15', '2021-07-27'))
ds1.attrs['plot_color']='green'
ds2.attrs['plot_color']='brown'
ds1.attrs['name']='Cendrosa obs'
ds2.attrs['name']='Els Plans obs'
ds_list= [ds1, ds2]

title='off'

#TS
mins=[0,-150,15,0]
maxs=[700,450,35,0.015]
for var in varlist:
    ylabel = var + ' (' + ds1[var].attrs['units'] + ')'
    # vmin=mins[varlist.index(var)]
    # vmax=maxs[varlist.index(var)]
    vmin,vmax=None, None
    time_series_ave(ds_list, var, 
                    # vmin=vmin, vmax=vmax,
                    title=title,
                    legend_out=None,
                    xlabel='Date',
                    ylabel=ylabel,
                    ds_colors=True,
                    figsize=(7, 4),
                    envelope=False,
                    # ds_linestyle=True
                    )
    if savefig:
        plt.savefig(f'figures/chap5/SOP_TS_DC/time_series_obs_{var}.png', dpi=300)


ds1.attrs['plot_color']='black'
ds2.attrs['plot_color']='black'
ds1.attrs['name']='obs'
ds2.attrs['name']='obs'

In [ ]:
savefig=False

varlist=['flat', 'sens', 't2m', 'q2m', 'SWdnSFC', 'LWdnSFC', 'psol','wind_speed_10m', 'wind_direction_10m']
varlist=['flat']

ds1=sop_cen_lmdz_routingfull_beta1
ds2=sop_els_lmdz_routingfull_beta1
#restrict to period between 15 and 27 July
# ds1=ds1.sel(time=slice('2021-07-15', '2021-07-27'))
# ds2=ds2.sel(time=slice('2021-07-15', '2021-07-27'))
ds1.attrs['plot_color']='green'
ds2.attrs['plot_color']='brown'
ds1.attrs['name']='Cendrosa'
ds2.attrs['name']='Els Plans'
ds_list= [ds1, ds2]

title='off'

#TS
mins=[0,-150,15,0]
maxs=[700,450,35,0.015]
for var in varlist:
    ylabel = var + ' (' + ds1[var].attrs['units'] + ')'
    # vmin=mins[varlist.index(var)]
    # vmax=maxs[varlist.index(var)]
    vmin,vmax=None, None
    time_series_ave(ds_list, var, 
                    # vmin=vmin, vmax=vmax,
                    title=title,
                    legend_out=None,
                    xlabel='Date',
                    ylabel=ylabel,
                    ds_colors=True,
                    figsize=(7, 4),
                    envelope=False,
                    # ds_linestyle=True
                    )
    if savefig:
        plt.savefig(f'figures/chap5/SOP_TS_DC/time_series_obs_{var}.png', dpi=300)

# ds1.attrs['plot_color']='black'
# ds2.attrs['plot_color']='black'
# ds1.attrs['name']='obs'
# ds2.attrs['name']='obs'

## Vertical profiles over both sites

In [ ]:
savefig=True

profiles_cendrosa=True
profiles_elsplans=True

ds1=iop_cen_lmdz_noirr
ds2=iop_cen_lmdz_irr100
ds3=iop_cen_lmdz_routingfull_beta1
ds4=iop_els_lmdz_noirr
ds5=iop_els_lmdz_irr100
ds6=iop_els_lmdz_routingfull_beta1
ds7= iop_cen_mesoNH_4D_cell_mean
ds8= iop_cen_mesoNH_4D_exact
ds9= iop_els_mesoNH_4D_cell_mean
ds10= iop_els_mesoNH_4D_exact
ds11= iop_cen_mesoNH_4D_cell_q25
ds12= iop_cen_mesoNH_4D_cell_q75
ds13= iop_els_mesoNH_4D_cell_q25
ds14= iop_els_mesoNH_4D_cell_q75

days=['1507','2007']
# days=['1507', '1607', '1707', '2007', '2107', '2207', '2707']
# times=[5.25,9.25,13.25, 17.25]
times=[12.25]

vars=['theta', 'ovap', 'wind_speed', 'wind_direction']
show_yticklabels_list=[True, False, False, False]
ylabels=[None, False, False, False]
vmins=[295, 0, 0, 0, None]
vmaxs=[312, 11, 10, 360, None]
xlabels=['Potential temperature (K)', 
         'Specific humidity (g kg⁻¹)', 
         'Wind speed (m s⁻¹)', 
         'Wind direction (°N)']
plot_quantiles=True

# vars=['wind_speed']#, 'wind_direction']
# vmins=[0, 0]
# vmaxs=[10, 360]
# xlabels=['Wind speed (m s⁻¹)', 'Wind direction (°)']
# plot_quantiles=False

altmax=3000

for i,var in enumerate(vars):
    xmin, xmax = vmins[i], vmaxs[i]
    xlabel = xlabels[i]
    ylabel=ylabels[i]
    show_yticklabels=show_yticklabels_list[i]
    for day in days:    
        if profiles_cendrosa:
            obs_dict=cen_RS[day]
            ds_list_lmdz = [ds1[day], ds2[day], ds3[day]]
            ds_list_mesoNH = [ds7[day], ds8[day]]
            quantiles, quantiles_color = [ds11[day], ds12[day]], 'orange'
            if not plot_quantiles:
                quantiles, quantiles_color = None, None
            profile_altitude_multipletimes_obs(ds_list_lmdz, obs_dict, var, times, 
                                            # title=title, 
                                            ds_list_mesoNH=ds_list_mesoNH,
                                            altmax=altmax, xmin=xmin, xmax=xmax, 
                                            # nbins=5,
                                            substract_gl=True,
                                            simfreq='30mn',
                                            xlabel=xlabel,
                                            ylabel=ylabel,
                                            quantiles=quantiles,
                                            quantiles_color=quantiles_color,
                                            show_yticklabels=show_yticklabels
                                            )
            if savefig:
                plt.savefig(f'figures/chap5/profiles/profile_cendrosa_{var}_{day}_.png', dpi=300, bbox_inches='tight')

        if profiles_elsplans:
            obs_dict=els_RS[day]
            ds_list_lmdz = [ds4[day], ds5[day], ds6[day]]
            ds_list_mesoNH = [ds9[day], ds10[day]]
            quantiles, quantiles_color = [ds13[day], ds14[day]], 'orange'
            if not plot_quantiles:
                quantiles, quantiles_color = None, None
            profile_altitude_multipletimes_obs(ds_list_lmdz, obs_dict, var, times, 
                                            # title=title, 
                                            ds_list_mesoNH=ds_list_mesoNH,
                                            altmax=altmax, xmin=xmin, xmax=xmax, 
                                            # nbins=5,
                                            substract_gl=True,
                                            simfreq='30mn',
                                            xlabel=xlabel,
                                            ylabel=ylabel,
                                            quantiles=quantiles,
                                            quantiles_color=quantiles_color,
                                            show_yticklabels=show_yticklabels
                                            )
            if savefig:
                plt.savefig(f'figures/chap5/profiles/profile_elsplans_{var}_{day}_.png', dpi=300, bbox_inches='tight')   

In [ ]:
#vertical profiles with mesoNH_sens subdatasets
savefig=False

profiles_cendrosa=False
profiles_elsplans=True

ds1=iop_cen_lmdz_noirr
ds2=iop_cen_lmdz_irr100
ds3=iop_cen_lmdz_routingfull_beta1
ds4=iop_els_lmdz_noirr
ds5=iop_els_lmdz_irr100
ds6=iop_els_lmdz_routingfull_beta1
ds7= iop_cen_mesoNH_4D_cell_mean
ds8= iop_cen_mesoNH_4D_exact
ds9= iop_els_mesoNH_4D_cell_mean
ds10= iop_els_mesoNH_4D_exact
ds11= iop_cen_mesoNH_4D_cell_q25
ds12= iop_cen_mesoNH_4D_cell_q75
ds13= iop_els_mesoNH_4D_cell_q25
ds14= iop_els_mesoNH_4D_cell_q75

# ds15 = mesoNH_cendrosa_1507_sens_mean
# ds15 = mesoNH_cendrosa_2007_sens_mean
# ds16 = mesoNH_elsplans_1507_sens_mean
# ds16 = mesoNH_elsplans_2007_sens_mean

ds15 = mesoNH_cendrosa_1507_flat_mean
ds16 = mesoNH_elsplans_1507_flat_mean
# ds15 = mesoNH_cendrosa_2007_flat_mean
# ds16 = mesoNH_elsplans_2007_flat_mean

days=['1507']
# days=['1507', '1607', '1707', '2007', '2107', '2207', '2707']
# times=[5.25,9.25,13.25, 17.25]
times=[12.25]

vars=['theta', 'ovap', 'wind_speed', 'wind_direction']
vmins=[300, 2, 0, 0, None]
vmaxs=[320, 14, 12, 360, None]
vmins=[295, 2, 0, 0, None]
vmaxs=[315, 14, 12, 360, None]
xlabels=['Potential temperature (K)', 
         'Specific humidity (g kg⁻¹)', 
         'Wind speed (m s⁻¹)', 
         'Wind direction (°)']
plot_quantiles=False

# vars=['wind_speed', 'wind_direction']
# vmins=[0, 0]
# vmaxs=[10, 360]
# xlabels=['Wind speed (m s⁻¹)', 'Wind direction (°)']
# plot_quantiles=False

altmax=3000

for i,var in enumerate(vars):
    xmin, xmax = vmins[i], vmaxs[i]
    xlabel = xlabels[i]
    for day in days:    
        if profiles_cendrosa:
            obs_dict=cen_RS[day]
            ds_list_lmdz = [ds1[day], ds2[day], ds3[day]]
            ds_list_mesoNH = [ds7[day], ds8[day], ds15]
            quantiles, quantiles_color = [ds11[day], ds12[day]], 'orange'
            if not plot_quantiles:
                quantiles, quantiles_color = None, None
            profile_altitude_multipletimes_obs(ds_list_lmdz, obs_dict, var, times, 
                                            # title=title, 
                                            ds_list_mesoNH=ds_list_mesoNH,
                                            altmax=altmax, xmin=xmin, xmax=xmax, 
                                            # nbins=5,
                                            substract_gl=True,
                                            simfreq='30mn',
                                            xlabel=xlabel,
                                            ylabel=False,
                                            quantiles=quantiles,
                                            quantiles_color=quantiles_color
                                            )
            if savefig:
                plt.savefig(f'figures/chap5/profiles/profile_cendrosa_{var}_{day}_.png', dpi=300, bbox_inches='tight')

        if profiles_elsplans:
            obs_dict=els_RS[day]
            ds_list_lmdz = [ds4[day], ds5[day], ds6[day]]
            ds_list_mesoNH = [ds9[day], ds10[day], ds16]
            quantiles, quantiles_color = [ds13[day], ds14[day]], 'orange'
            if not plot_quantiles:
                quantiles, quantiles_color = None, None
            profile_altitude_multipletimes_obs(ds_list_lmdz, obs_dict, var, times, 
                                            # title=title, 
                                            ds_list_mesoNH=ds_list_mesoNH,
                                            altmax=altmax, xmin=xmin, xmax=xmax, 
                                            # nbins=5,
                                            substract_gl=True,
                                            simfreq='30mn',
                                            xlabel=xlabel,
                                            ylabel=False,
                                            quantiles=quantiles,
                                            quantiles_color=quantiles_color
                                            )
            if savefig:
                plt.savefig(f'figures/chap5/profiles/profile_elsplans_{var}_{day}_.png', dpi=300, bbox_inches='tight')   

## Maps

In [ ]:
lon_min_mesoNH=-3.4
lon_max_mesoNH=4
lat_min_mesoNH=40.5
lat_max_mesoNH=43.5

lat_min_mesoNH, lat_max_mesoNH, lon_min_mesoNH, lon_max_mesoNH = 41.2, 42., 0., 1.5

figsize_mesoNH=(10.5, 4)

In [ ]:
#load LMDZ mappable file
sim='sim2931'
lmdz_routingfull_beta1_mappable_file = '{}/{}_20210701_20210731_HF_histhf.nc'.format(boost_irrig_dir, sim)
lmdz_routingfull_beta1_mappable = format_lmdz_HF(lmdz_routingfull_beta1_mappable_file, color='green', name='irr_boost')
lmdz_routingfull_beta1_mappable = lmdz_routingfull_beta1_mappable.sel(time=slice('2021-07-14', '2021-07-30'))   

vars=['temp', 'geoph', 'vitu', 'vitv']
levels=[85000, 70000, 50000]
for var in vars:
    for level in levels:
        lmdz_routingfull_beta1_mappable = extract_var_presniv(lmdz_routingfull_beta1_mappable, var, level)
rename_dict={   'vitu_850':'u850',
                'vitv_850':'v850',
                'vitu_700':'u700',
                'vitv_700':'v700',
                'vitu_500':'u500',
                'vitv_500':'v500',
            }
lmdz_routingfull_beta1_mappable = lmdz_routingfull_beta1_mappable.rename(rename_dict)
lmdz_routingfull_beta1_mappable

### Average

In [ ]:
datemin='2021-07-15'
datemax='2021-07-27'

mesoNH_sop=mesoNH.sel(time=slice(datemin, datemax))
lmdz_routingfull_beta1_mappable_sop=lmdz_routingfull_beta1_mappable.sel(time=slice(datemin, datemax))

In [ ]:
var='flat'
ds1=lmdz_routingfull_beta1_mappable_sop
lon_min=lon_min_mesoNH
lon_max=lon_max_mesoNH
lat_min=lat_min_mesoNH
lat_max=lat_max_mesoNH
cmap=bluesW
vmin=0
vmax=200
# vmin,vmax=None, None
ds1=ds1.sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))
map_ave(ds1, var, figsize=figsize_mesoNH,
        cmap=cmap,
        vmin=vmin,
        vmax=vmax
        )
add_liaise_site_loc()

map_mesoNH_mean(mesoNH_sop, var, add_liaise=True, vmin=vmin, vmax=vmax, cmap=cmap,)

map_mesoNH_mean_restrict(mesoNH_sop, var, add_liaise=True, vmin=vmin, vmax=vmax, cmap=cmap,)


In [ ]:
var='LWdnSFC'
vmin=200
vmax=400
# vmin,vmax=None, None
# cmap=myvir

In [ ]:
ds1=lmdz_routingfull_beta1_mappable
# lon_min=lon_min_mesoNH
# lon_max=lon_max_mesoNH
# lat_min=lat_min_mesoNH
# lat_max=lat_max_mesoNH
lon_min=-10
lon_max=8
lat_min=36
lat_max=48
cmap=reds
vmin=0
vmax=17000
vmin,vmax=None, None
ds1=ds1.sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))
map_wind(ds1, 
         extra_var='geoph_500', extra_ds=ds1,
        figsize=figsize_mesoNH,  cmap=cmap,  vmin=vmin, vmax=vmax,
        height='500',
        dist=4,
        scale=100,
        )
add_liaise_site_loc()

### Timestamp

In [ ]:
timestamp='2021-07-20T12:00:00'
timestamps=['2021-07-15T12:00:00', '2021-07-16T12:00:00', '2021-07-17T12:00:00', '2021-07-20T12:00:00', '2021-07-21T12:00:00', '2021-07-22T12:00:00', '2021-07-27T12:00:00']

In [ ]:
var='flat'
vmin=0
vmax=500
# vmin,vmax=None, None
cmap=bluesW
map_mesoNH_timestamp_restrict(mesoNH, var, add_liaise=True, timestamp=timestamp,
                              vmin=vmin, vmax=vmax, cmap=cmap,
                              poly=both_cells,)

ds1=lmdz_routingfull_beta1_mappable
ds1=ds1.sel(time=timestamp, method='nearest')
ds1=ds1.sel(lon=slice(lon_min_mesoNH, lon_max_mesoNH), lat=slice(lat_min_mesoNH, lat_max_mesoNH))
map_ave(ds1,var, 
        vmin=vmin, vmax=vmax, cmap=cmap,
        figsize=(10.5,4),
        # add_liaise=True,
)
add_liaise_site_loc()

In [ ]:
ds1=lmdz_routingfull_beta1_mappable
timestamp='2021-07-20T12:00:00'
#restrict to timestamp
ds1=ds1.sel(time=timestamp, method='nearest')
ds1

In [ ]:
var='cldt'
ds1=lmdz_routingfull_beta1_mappable
lon_min=lon_min_mesoNH
lon_max=lon_max_mesoNH
lat_min=lat_min_mesoNH
lat_max=lat_max_mesoNH
# lon_min=-10
# lon_max=8
# lat_min=36
# lat_max=48
cmap=greys
vmin=0
vmax=50
# vmin,vmax=None, None
ds1=ds1.sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))
for timestamp in timestamps:
    ds2=ds1.sel(time=timestamp, method='nearest')
    map_ave(ds2, var, 
            figsize=figsize_mesoNH,  cmap=cmap,  vmin=vmin, vmax=vmax,
            )
    add_liaise_site_loc()

In [ ]:
ds1=lmdz_routingfull_beta1_mappable
lon_min=lon_min_mesoNH
lon_max=lon_max_mesoNH
lat_min=lat_min_mesoNH
lat_max=lat_max_mesoNH
# lon_min=-10
# lon_max=8
# lat_min=36
# lat_max=48
cmap=reds
vmin=1500
vmax=3100
# vmin,vmax=None, None
ds1=ds1.sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))
for timestamp in timestamps:
    ds2=ds1.sel(time=timestamp, method='nearest')
    map_wind(ds2, 
            extra_var='geoph_850', extra_ds=ds2,
            figsize=figsize_mesoNH,  cmap=cmap,  vmin=vmin, vmax=vmax,
            height='850',
            dist=2,
            scale=50,
            )
    add_liaise_site_loc()

## SOP time series

In [ ]:
savefig=True
start_date='2021-07-15'
end_date='2021-07-15'

varlist, plot_envelope=['flat'], True
varlist, plot_envelope=['SWdnSFC', 'LWdnSFC', 'flat', 'sens', 't2m','q2m', 'wind_speed_10m', 'wind_direction_10m'], True
# varlist, plot_envelope=['wind_speed_10m', 'wind_direction_10m'], True
vmins=[0,300,-50, -200, 12, 6, 0, 0]
vmaxs=[1000,440, 600, 500, 37, 14, 9, 360]

ds1= cen_lmdz_noirr.sel(time=slice(start_date, end_date))
ds2= cen_lmdz_irr100.sel(time=slice(start_date, end_date))
ds3= cen_lmdz_routingfull_beta1.sel(time=slice(start_date, end_date))
ds4=mesoNH_cendrosa_cell.sel(time=slice(start_date, end_date))
ds4.attrs['show_envelope'] = True
ds5=mesoNH_cendrosa_exact.sel(time=slice(start_date, end_date))
ds6=restrict_cen_mto.sel(time=slice(start_date, end_date))
ds_list=[ds6, ds1, ds2, ds3, ds5, ds4]

title='off'

#TS
for var in varlist:
    ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
    vmin=vmins[varlist.index(var)]
    vmax=vmaxs[varlist.index(var)]
    # vmin,vmax=None, None
    time_series_ave(ds_list, var, 
                    vmin=vmin, vmax=vmax,
                    title=title,
                    legend_out=None,
                    xlabel='Time',
                    ylabel=ylabel,
                    ds_colors=True,
                    figsize=(7, 4),
                    envelope=plot_envelope
                    )
    if savefig:
        plt.savefig(f'figures/chap5/IOP_TS/TS_{start_date}_cendrosa_{var}.png', dpi=300)


In [ ]:
savefig=True
start_date='2021-07-20'
end_date='2021-07-20'
vmins=[0,300,-50, -200, 12, 6, 0, 0]
vmaxs=[1000,440, 600, 500, 37, 14, 9, 360]

varlist, plot_envelope=['flat'], True
varlist, plot_envelope=['SWdnSFC', 'LWdnSFC','flat', 'sens', 't2m','q2m', 'wind_speed_10m', 'wind_direction_10m'], True

ds1= cen_lmdz_noirr.sel(time=slice(start_date, end_date))
ds2= cen_lmdz_irr100.sel(time=slice(start_date, end_date))
ds3= cen_lmdz_routingfull_beta1.sel(time=slice(start_date, end_date))
ds4=mesoNH_cendrosa_cell.sel(time=slice(start_date, end_date))
ds4.attrs['show_envelope'] = True
ds5=mesoNH_cendrosa_exact.sel(time=slice(start_date, end_date))
ds6=restrict_cen_mto.sel(time=slice(start_date, end_date))
ds_list= [ds6, ds1, ds2, ds3, ds5, ds4]  

title='off'

#TS
for var in varlist:
    ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
    vmin=vmins[varlist.index(var)]
    vmax=vmaxs[varlist.index(var)]
    # vmin,vmax=None, None
    time_series_ave(ds_list, var, 
                    vmin=vmin, vmax=vmax,
                    title=title,
                    legend_out=None,
                    xlabel='Time',
                    ylabel=ylabel,
                    ds_colors=True,
                    figsize=(7, 4),
                    envelope=plot_envelope
                    )
    if savefig:
        plt.savefig(f'figures/chap5/IOP_TS/TS_{start_date}_cendrosa_{var}.png', dpi=300)


In [ ]:
savefig=True
start_date='2021-07-15'
end_date='2021-07-15'

varlist=['flat'], True
varlist=['SWdnSFC', 'LWdnSFC', 'flat', 'sens', 't2m','q2m', 'wind_speed_10m', 'wind_direction_10m']
plot_envelope=True

vmins=[0,       300,    -20,    -20,    12,     6,      0,  0]
vmaxs=[1000,    440,    100,    520,    37,     14,     10,  360]

ds1= els_lmdz_noirr.sel(time=slice(start_date, end_date))
ds2= els_lmdz_irr100.sel(time=slice(start_date, end_date))
ds3= els_lmdz_routingfull_beta1.sel(time=slice(start_date, end_date))
ds4=mesoNH_elsplans_cell.sel(time=slice(start_date, end_date))
ds4.attrs['show_envelope'] = True
ds5=mesoNH_elsplans_exact.sel(time=slice(start_date, end_date))
ds6=restrict_els_mto.sel(time=slice(start_date, end_date))
ds_list=[ds6, ds1, ds2, ds3, ds5, ds4]

title='off'

#TS
for var in varlist:
    ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
    vmin=vmins[varlist.index(var)]
    vmax=vmaxs[varlist.index(var)]
    # vmin,vmax=None, None
    time_series_ave(ds_list, var, 
                    vmin=vmin, vmax=vmax,
                    title=title,
                    legend_out=None,
                    xlabel='Time',
                    ylabel=ylabel,
                    ds_colors=True,
                    figsize=(7, 4),
                    envelope=plot_envelope
                    )
    if savefig:
        plt.savefig(f'figures/chap5/IOP_TS/TS_{start_date}_elsplans_{var}.png', dpi=300)


In [ ]:
savefig=True
start_date='2021-07-20'
end_date='2021-07-20'

varlist=['flat'], True
varlist=['SWdnSFC', 'LWdnSFC', 'flat', 'sens', 't2m','q2m', 'wind_speed_10m', 'wind_direction_10m']
plot_envelope=True

vmins=[0,       300,    -20,    -20,    12,     6,      0,  0]
vmaxs=[1000,    440,    100,    520,    37,     14,     10,  360]

ds1= els_lmdz_noirr.sel(time=slice(start_date, end_date))
ds2= els_lmdz_irr100.sel(time=slice(start_date, end_date))
ds3= els_lmdz_routingfull_beta1.sel(time=slice(start_date, end_date))
ds4=mesoNH_elsplans_cell.sel(time=slice(start_date, end_date))
ds4.attrs['show_envelope'] = True
ds5=mesoNH_elsplans_exact.sel(time=slice(start_date, end_date))
ds6=restrict_els_mto.sel(time=slice(start_date, end_date))
ds_list=[ds6, ds1, ds2, ds3, ds5, ds4]

title='off'

#TS
for var in varlist:
    ylabel = ds1[var].attrs['long_name'] + ' (' + ds1[var].attrs['units'] + ')'
    vmin=vmins[varlist.index(var)]
    vmax=vmaxs[varlist.index(var)]
    # vmin,vmax=None, None
    time_series_ave(ds_list, var, 
                    vmin=vmin, vmax=vmax,
                    title=title,
                    legend_out=None,
                    xlabel='Time',
                    ylabel=ylabel,
                    ds_colors=True,
                    figsize=(7, 4),
                    envelope=plot_envelope
                    )
    if savefig:
        plt.savefig(f'figures/chap5/IOP_TS/TS_{start_date}_elsplans_{var}.png', dpi=300)


### Time series over the two days

## Bins and distribution

In [ ]:
savefig=True
timestamps=['2021-07-15T12:00:00', '2021-07-20T12:00:00']
xmin=0
xmax=550
ylim=(0,52)
nbins=22
title='off'
for timestamp in timestamps:
    ds=mesoNH_cendrosa_cell
    ds_list=[mesoNH_cendrosa_exact, cen_lmdz_noirr, cen_lmdz_irr100, cen_lmdz_routingfull_beta1, cen_mto]
    var='sens'
    xlabel='Sensible heat flux (W m⁻²)'
    bins_timestamp(ds, var, timestamp, nbins=nbins, title=title, xlabel=xlabel, xmin=xmin, xmax=xmax, ylim=ylim, ds_list=ds_list, site='- La Cendrosa')
    if savefig:
        plt.savefig(f'figures/chap5/IOP_bins/bins_{var}_{timestamp}_cendrosa.png', dpi=300, bbox_inches='tight')
    var='flat'
    xlabel='Latent heat flux (W m⁻²)'
    bins_timestamp(ds, var, timestamp, nbins=nbins, title=title, xlabel=xlabel, xmin=xmin, xmax=xmax, ylim=ylim, ds_list=ds_list, site='- La Cendrosa')
    if savefig:
        plt.savefig(f'figures/chap5/IOP_bins/bins_{var}_{timestamp}_cendrosa.png', dpi=300, bbox_inches='tight')

    ds=mesoNH_elsplans_cell
    ds_list=[mesoNH_elsplans_exact, els_lmdz_noirr, els_lmdz_irr100, els_lmdz_routingfull_beta1, els_mto]
    var='sens'
    xlabel='Sensible heat flux (W m⁻²)'
    bins_timestamp(ds, var, timestamp, nbins=nbins, title=title, xlabel=xlabel, xmin=xmin, xmax=xmax, ylim=ylim, ds_list=ds_list, site='- Els Plans')
    if savefig:
        plt.savefig(f'figures/chap5/IOP_bins/bins_{var}_{timestamp}_elsplans.png', dpi=300, bbox_inches='tight')
    var='flat'
    xlabel='Latent heat flux (W m⁻²)'
    bins_timestamp(ds, var, timestamp, nbins=nbins, title=title, xlabel=xlabel, xmin=xmin, xmax=xmax, ylim=ylim, ds_list=ds_list, site='- Els Plans')
    if savefig:
        plt.savefig(f'figures/chap5/IOP_bins/bins_{var}_{timestamp}_elsplans.png', dpi=300, bbox_inches='tight')